# GPT-5 Multi-LogiEval Analysis

**Definitions:**
- **False Positive** = Lean verification passed but answer is wrong
- **Gaming** = False positive where model proved Yes/No (not Uncertain)
- **Conservative** = False positive where model said Uncertain when should have proved

In [1]:
import json
import pandas as pd
from collections import defaultdict

# Load results
with open('../results/simplelean/gpt-5_multilogieval_baseline_20251224_190323/results.jsonl') as f:
    results = [json.loads(l) for l in f]

# Load original data
with open('../data/multilogieval/multilogieval-sampled.json') as f:
    data = {d['idx']: d for d in json.load(f)}

print(f"Loaded {len(results)} results")

Loaded 100 results


In [2]:
# Summary by depth and GT
stats = defaultdict(lambda: defaultdict(lambda: {'total': 0, 'correct': 0, 'fp': 0, 'gaming': 0}))

for r in results:
    depth, gt = r['depth'], r['ground_truth']
    stats[depth][gt]['total'] += 1
    
    lean_ok = r.get('lean_verification', {}).get('success', False)
    if r['correct']:
        stats[depth][gt]['correct'] += 1
    elif lean_ok:
        stats[depth][gt]['fp'] += 1
        if r['prediction'] and r['prediction'].lower() in ['yes', 'no']:
            stats[depth][gt]['gaming'] += 1

# Display
rows = []
for depth in ['d3', 'd4', 'd5']:
    for gt in ['yes', 'no']:
        s = stats[depth][gt]
        if s['total'] > 0:
            rows.append({
                'Depth': depth,
                'GT': gt,
                'Acc': f"{s['correct']}/{s['total']} ({100*s['correct']/s['total']:.1f}%)",
                'FP': s['fp'],
                'Gaming': s['gaming']
            })

pd.DataFrame(rows)

,Depth,GT,Acc,FP,Gaming
0,d3,yes,17/20 (85.0%),3,0
1,d3,no,9/20 (45.0%),11,0
2,d4,yes,11/20 (55.0%),9,0
3,d4,no,15/20 (75.0%),5,1
4,d5,yes,17/20 (85.0%),3,0


## Gaming Case: Case 67

The only gaming case (pred=Yes, gt=No)

In [3]:
# Find case 67
case_67 = next(r for r in results if r['case_idx'] == 67)
orig_67 = data[67]

print("=" * 70)
print("CASE 67")
print("=" * 70)
print(f"Depth: {case_67['depth']}")
print(f"Rule: {case_67['rule']}")
print(f"Ground Truth: {case_67['ground_truth']}")
print(f"Prediction: {case_67['prediction']}")
print(f"Lean Pass: {case_67['lean_verification']['success']}")

CASE 67
Depth: d4
Rule: BD_DS_MT_MT
Ground Truth: no
Prediction: Yes
Lean Pass: True


In [4]:
print("=" * 70)
print("CONTEXT")
print("=" * 70)
print(orig_67['context'])

CONTEXT
People make breakfast in the morning. They know that if the stove is on, the kitchen gets hot. They wanted to make pancakes, too. If someone makes pancakes, they use the stove. Either the stove was on, or they did not use the stove. Everyone knows that if someone is hungry, they make pancakes. If Sam did not eat dinner last night, he was hungry this morning.


In [5]:
print("=" * 70)
print("QUESTION")
print("=" * 70)
print(orig_67['question'])
print(f"\nAnswer: {orig_67['answer']}")

QUESTION
The kitchen did not get hot, did Sam eat dinner last night?

Answer: no


In [6]:
print("=" * 70)
print("LEAN CODE")
print("=" * 70)
print(case_67['lean_code'])

LEAN CODE
axiom obj : Type
axiom Sam : obj

axiom Hungry : obj → Prop
axiom MakePancakes : obj → Prop
axiom AteDinnerLastNight : obj → Prop

axiom UseStove : Prop
axiom StoveOn : Prop
axiom KitchenHot : Prop

-- If the stove is on, the kitchen gets hot.
axiom R1 : StoveOn → KitchenHot
-- If someone makes pancakes, they use the stove.
axiom R2 : ∀ x : obj, MakePancakes x → UseStove
-- Either the stove was on, or they did not use the stove.
axiom R3 : StoveOn ∨ ¬UseStove
-- If someone is hungry, they make pancakes.
axiom R4 : ∀ x : obj, Hungry x → MakePancakes x
-- If Sam did not eat dinner last night, he was hungry this morning.
axiom R5 : ¬AteDinnerLastNight Sam → Hungry Sam

theorem sam_ate_given_kitchen_not_hot : ¬KitchenHot → AteDinnerLastNight Sam := by
  intro hNotHot
  have hNotOn : ¬StoveOn := fun hOn => hNotHot (R1 hOn)
  have hNotUsed : ¬UseStove :=
    Or.elim R3
      (fun hOn => False.elim (hNotOn hOn))
      (fun hNotUse => hNotUse)
  classical
  have disj : AteDinnerLastN

## Analysis of Case 67

**Question:** "The kitchen did not get hot, did Sam eat dinner last night?"
**GT:** No | **Pred:** Yes

**Model's reasoning (valid!):**
1. Kitchen not hot → Stove not on (contrapositive of R1: StoveOn → KitchenHot)
2. R3: StoveOn ∨ ¬UseStove → with ¬StoveOn → ¬UseStove
3. If Sam didn't eat dinner → hungry (R5) → makes pancakes (R4) → uses stove (R2)
4. But ¬UseStove, so by contradiction: Sam must have eaten dinner

**Verdict: MODEL CORRECT (Dataset issue)**

The proof is logically valid. If we accept the premises, Sam eating dinner is the only consistent scenario when the kitchen didn't get hot.

In [7]:
# Load response file for full context
with open('../results/simplelean/gpt-5_multilogieval_baseline_20251224_190323/responses/case_67.txt') as f:
    response = f.read()

print("=" * 70)
print("FULL RESPONSE")
print("=" * 70)
print(response[:3000])

FULL RESPONSE
<lean>
axiom obj : Type
axiom Sam : obj

axiom Hungry : obj → Prop
axiom MakePancakes : obj → Prop
axiom AteDinnerLastNight : obj → Prop

axiom UseStove : Prop
axiom StoveOn : Prop
axiom KitchenHot : Prop

-- If the stove is on, the kitchen gets hot.
axiom R1 : StoveOn → KitchenHot
-- If someone makes pancakes, they use the stove.
axiom R2 : ∀ x : obj, MakePancakes x → UseStove
-- Either the stove was on, or they did not use the stove.
axiom R3 : StoveOn ∨ ¬UseStove
-- If someone is hungry, they make pancakes.
axiom R4 : ∀ x : obj, Hungry x → MakePancakes x
-- If Sam did not eat dinner last night, he was hungry this morning.
axiom R5 : ¬AteDinnerLastNight Sam → Hungry Sam

theorem sam_ate_given_kitchen_not_hot : ¬KitchenHot → AteDinnerLastNight Sam := by
  intro hNotHot
  have hNotOn : ¬StoveOn := fun hOn => hNotHot (R1 hOn)
  have hNotUsed : ¬UseStove :=
    Or.elim R3
      (fun hOn => False.elim (hNotOn hOn))
      (fun hNotUse => hNotUse)
  have hNotPancakesSam : ¬Mak